In [1]:
import gc

# import modin.pandas as modin # find a different name :/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# from modin import config
# config.Engine.put("dask")
# config.StorageFormat.put("pandas")

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

In [2]:
fred = pd.read_csv(r"./data/FRED/FRED3_Entire_Database_2021.csv", low_memory=False, 
                   header=0, skiprows=range(1, 10)) # has annotations embedded inside the csv :/
meta = pd.read_csv(r"./data/FRED/FRED3_Column_Definitions_2021.csv", low_memory=False)
# groot = pd.read_csv(r"./GRooTFullVersion.csv", low_memory=False, encoding="latin")

In [3]:
fred.shape, meta.shape   # , groot.shape

((57190, 1164), (1195, 13))

In [4]:
meta.loc[:, ["column_id", "name", "units"]]

,column_id,name,units
0,F00001,Version_FRED version number,NaN
1,F00002,Notes_Row ID,NaN
2,F00003,Abbreviated article citation,NaN
3,F00004,Data source_Citation,NaN
4,F00005,Data source_DOI,NaN
...,...,...,...
1190,F01470,n_Soil Ca_Exchangeable Ca2+ mass,NaN
1191,F01471,n_Soil K content,NaN
1192,F01472,SE_Soil K content,mg/g
1193,F01473,n_Soil Mg_Exchangeable Mg2+ mass,NaN


In [5]:
lookup = pd.Series(data=meta.name.values, index=meta.column_id)
lookup

column_id
F00001         Version_FRED version number
F00002                        Notes_Row ID
F00003        Abbreviated article citation
F00004                Data source_Citation
F00005                     Data source_DOI
                        ...               
F01470    n_Soil Ca_Exchangeable Ca2+ mass
F01471                    n_Soil K content
F01472                   SE_Soil K content
F01473    n_Soil Mg_Exchangeable Mg2+ mass
F01474     n_Soil Na_Exchangeable Na+ mass
Length: 1195, dtype: object

In [6]:
lookup[lookup.str.contains("order")]

column_id
F00056                                           Root order
F00057                                       Min_Root order
F00058                                       Max_Root order
F00059               Notes_Root order classification scheme
F00179    Root branching intensity (branching ratio)_Num...
F00180    n_Root branching intensity (branching ratio)_N...
F00181    SE_Root branching intensity (branching ratio)_...
F00182    Root branching intensity_root tips per higher ...
F00184    SE_Root branching intensity_root tips per high...
F00185    Min_Root branching intensity_root tips per hig...
F00186    Max_Root branching intensity_root tips per hig...
F00187    Root branching architecture_Root length per hi...
F00858              Root mass fraction per root order class
F00859            n_Root mass fraction per root order class
F00860           SE_Root mass fraction per root order class
F00867            Root length fraction per root order class
F00868          n_Root length 

In [21]:
colnames = pd.Series([lookup[cname] for cname in fred.columns], dtype=str)

In [22]:
colnames

0                             Version_FRED version number
1                                            Notes_Row ID
2                            Abbreviated article citation
3                                    Data source_Citation
4                                         Data source_DOI
                              ...                        
1159    Aboveground growth_Annual net primary producti...
1160    n_Aboveground growth_Annual net primary produc...
1161    SE_Aboveground growth_Annual net primary produ...
1162                      Aboveground annual C production
1163                            Notes_Miscellaneous notes
Length: 1164, dtype: object

In [23]:
# FRED is a data wrangling disaster
colnames[colnames.str.contains("root")]

40                Notes_Method of determining dead roots
66       Root passage cell number per root circumference
79               Root stele diameter/root diameter ratio
80             n_Root stele diameter/root diameter ratio
81            SE_Root stele diameter/root diameter ratio
82                          Root stele/root cortex ratio
83                        n_Root stele/root cortex ratio
84                       SE_Root stele/root cortex ratio
85               Root conduit number per root stele area
86             n_Root conduit number per root stele area
87            SE_Root conduit number per root stele area
119     Root vessel number per root cross-sectional area
120    n_Root vessel number per root cross-sectional ...
122                    n_root xylem cross-sectional area
123                   SE_root xylem cross-sectional area
127         Root xylem vessel number per root stele area
128       n_Root xylem vessel number per root stele area
129      SE_Root xylem vessel n

In [24]:
# get rid of the embedded metadata
fred.iloc[9:, :]

F00001 F00002           F00003  \
9          1      1  Long et al 2013   
10         1      2  Long et al 2013   
11         1      3  Long et al 2013   
12         1      4  Long et al 2013   
13         1      5  Long et al 2013   
...      ...    ...              ...   
57194      3  57516   Lin et al 2020   
57195      3  57517   Lin et al 2020   
57196      3  57518   Lin et al 2020   
57197      3  57519   Lin et al 2020   
57198      3  57520   Lin et al 2020   

                                                  F00004  \
9      Long Y, Kong D, Chen Z, Zeng H. 2013. Variatio...   
10     Long Y, Kong D, Chen Z, Zeng H. 2013. Variatio...   
11     Long Y, Kong D, Chen Z, Zeng H. 2013. Variatio...   
12     Long Y, Kong D, Chen Z, Zeng H. 2013. Variatio...   
13     Long Y, Kong D, Chen Z, Zeng H. 2013. Variatio...   
...                                                  ...   
57194  Lin D, Yang S, Dou P, Wang H, Wang F, Qian S, ...   
57195  Lin D, Yang S, Dou P, Wang H, Wang F, Qian S, ...   
57196  Lin D, Yang S, Dou P, Wang H, Wang F, Qian S, ...   
57197  Lin D, Yang S, Dou P, Wang H, Wang F, Qian S, ...   
57198  Lin D, Yang S, Dou P, Wang H, Wang F, Qian S, ...   

                             F00005 F00006 F00007 F00008       F00009 F00010  \
9      10.1371/journal.pone.0057153    NaN    NaN      1          NaN    NaN   
10     10.1371/journal.pone.0057153    NaN    NaN      1          NaN    NaN   
11     10.1371/journal.pone.0057153    NaN    NaN      1          NaN    NaN   
12     10.1371/journal.pone.0057153    NaN    NaN      1          NaN    NaN   
13     10.1371/journal.pone.0057153    NaN    NaN      1          NaN    NaN   
...                             ...    ...    ...    ...          ...    ...   
57194            10.1093/aob/mcz166    NaN    NaN    NaN  contributed      x   
57195            10.1093/aob/mcz166    NaN    NaN    NaN  contributed      x   
57196            10.1093/aob/mcz166    NaN    NaN    NaN  contributed      x   
57197            10.1093/aob/mcz166    NaN    NaN    NaN  contributed      x   
57198            10.1093/aob/mcz166    NaN    NaN    NaN  contributed      x   

              F01291         F01290          F01289          F01286  \
9      Pteridophytes  Gleicheniales  Gleicheniaceae   Dicranopteris   
10     Pteridophytes  Gleicheniales  Gleicheniaceae   Dicranopteris   
11     Pteridophytes  Gleicheniales  Gleicheniaceae   Dicranopteris   
12       Gymnosperms        Pinales    Cupressaceae    Cunninghamia   
13       Gymnosperms        Pinales    Cupressaceae    Cunninghamia   
...              ...            ...             ...             ...   
57194  Pteridophytes  Gleicheniales  Gleicheniaceae  Diplopterygium   
57195  Pteridophytes  Gleicheniales  Gleicheniaceae  Diplopterygium   
57196  Pteridophytes    Osmundaceae     Osmundaceae         Osmunda   
57197  Pteridophytes    Osmundaceae     Osmundaceae         Osmunda   
57198  Pteridophytes    Osmundaceae     Osmundaceae         Osmunda   

           F01287 F01344          F00017          F00018      F00019 F00020  \
9        linearis    NaN  Gleicheniaceae   Dicranopteris   dichotoma    NaN   
10       linearis    NaN  Gleicheniaceae   Dicranopteris   dichotoma    NaN   
11       linearis    NaN  Gleicheniaceae   Dicranopteris   dichotoma    NaN   
12     lanceolata    NaN    Cupressaceae    Cunninghamia  lanceolata    NaN   
13     lanceolata    NaN    Cupressaceae    Cunninghamia  lanceolata    NaN   
...           ...    ...             ...             ...         ...    ...   
57194   chinensis    NaN             NaN  Diplopterygium    chinense    NaN   
57195   chinensis    NaN             NaN  Diplopterygium    chinense    NaN   
57196    japonica    NaN             NaN         Osmunda    japonica    NaN   
57197    japonica    NaN             NaN         Osmunda    japonica    NaN   
57198    japonica    NaN             NaN         Osmunda    japonica    NaN   

      F01413          F00027 F00032      

In [14]:
# fred = fred.rename(columns={id : desc for (id, desc) in zip(lookup.index, lookup.values)}).iloc[9:, :]

In [15]:
fred.iloc[9:, 1]

18          10
19          11
20          12
21          13
22          14
         ...  
57194    57516
57195    57517
57196    57518
57197    57519
57198    57520
Name: Notes_Row ID, Length: 57181, dtype: object

In [31]:
fred.query("F00002 == 53904")

,F00001,F00002,F00003,F00004,F00005,F00006,F00007,F00008,F00009,F00010,F01291,F01290,F01289,F01286,F01287,F01344,F00017,F00018,F00019,F00020,F01413,F00027,F00032,F01456,F00042,F00041,F00044,F00043,F00055,F00056,F00057,F00058,F00059,F00060,F00949,F00950,F00061,F00062,F00063,F00064,F00065,F00066,F00067,F00068,F00069,F00072,F00073,F00074,F00075,F00097,F00098,F01374,F01375,F01376,F01377,F01378,F00100,F00101,F00102,F00103,F00104,F00105,F00106,F00107,F00108,F00109,F00110,F00111,F00112,F00113,F00114,F00157,F00122,F00123,F00124,F00118,F00119,F00120,F00121,F00115,F00116,F00117,F00125,F00126,F00127,F00134,F00135,F00136,F00142,F00143,F00144,F00154,F00155,F00156,F00150,F00151,F00128,F00129,F00140,F00141,F00132,F00133,F01313,F01314,F01327,F00152,F00153,F00130,F00131,F00148,F00149,F01324,F01325,F01319,F01320,F01326,F00145,F00146,F00147,F01321,F01322,F01311,F01312,F01448,F01315,F01316,F01449,F00137,F00138,F00139,F00187,F00179,F00180,F00181,F00182,F01415,F00184,F00185,F00186,F01371,F01372,F01373,F01339,F01340,F01341,F01342,F01343,F00219,F00220,F00221,F00216,F00218,F00222,F00223,F00176,F00177,F00178,F00224,F00225,F00192,F00193,F00194,F00195,F00210,F00211,F00212,F00213,F00214,F00215,F01414,F00207,F00208,F00209,F00199,F00170,F00171,F00172,F00203,F00204,F00205,F00206,F00173,F00174,F00175,F00226,F00253,F00254,F00255,F00256,F00249,F00250,F00251,F00252,F01299,F00289,F00290,F00291,F00292,F01302,F00257,F00258,F00259,F00260,F01300,F00273,F00261,F00262,F00263,F00264,F00265,F00266,F00267,F00268,F00269,F00270,F00271,F00272,F00276,F00277,F00278,F00279,F00280,F00281,F00282,F00283,F00284,F00285,F01301,F00293,F00294,F00295,F01303,F00296,F01338,F00297,F01304,F00298,F00299,F00300,F00301,F00302,F00303,F00304,F00305,F00306,F00307,F00308,F00310,F00311,F00312,F00313,F00314,F00315,F00316,F00317,F00318,F00322,F00323,F00324,F00325,F01305,F00334,F01388,F00331,F00333,F00326,F00327,F00328,F00319,F00320,F00321,F00329,F00330,F00335,F00336,F00337,F00246,F00247,F00248,F00245,F00237,F00238,F00239,F00240,F00242,F00243,F01389,F00244,F00380,F00381,F00342,F00343,F00344,F00345,F00431,F00432,F00433,F00346,F00437,F00438,F00439,F00340,F00347,F01446,F00348,F00349,F00350,F00351,F01330,F01331,F01332,F00352,F00353,F00354,F00355,F00358,F00359,F00360,F00361,F00356,F00357,F00338,F00339,F00362,F01391,F01387,F00364,F00365,F00366,F00367,F00368,F00369,F00374,F00375,F00376,F00377,F00434,F00435,F00436,F00378,F00379,F00382,F01239,F00383,F01328,F00384,F00385,F00386,F00387,F00388,F00389,F00390,F00391,F01329,F00392,F00393,F00394,F00395,F00396,F00397,F00398,F00399,F00400,F00401,F01281,F01450,F01451,F00402,F00407,F00408,F00403,F00404,F00405,F00406,F00413,F00414,F00415,F00416,F00417,F00418,F00419,F01390,F00420,F00409,F00410,F00411,F00412,F00421,F00422,F00423,F00424,F00425,F00426,F00427,F00428,F00429,F00430,F00461,F00462,F00466,F00467,F00468,F00465,F00457,F01364,F01365,F00458,F00459,F00460,F00591,F00592,F00593,F00463,F00464,F00077,F01285,F00085,F00476,F00477,F00478,F00469,F01295,F01292,F00470,F00471,F00472,F00473,F00474,F00475,F01296,F01293,F00483,F00484,F00480,F00481,F00482,F00479,F00518,F00519,F00520,F00546,F00547,F00548,F00549,F00550,F00553,F00554,F00521,F00522,F00523,F00524,F00525,F00510,F00511,F00571,F00572,F00573,F01358,F00559,F00560,F00543,F00544,F00545,F00527,F00528,F00526,F00507,F00508,F00509,F00538,F00539,F00568,F00551,F00552,F00569,F00529,F01420,F01421,F01422,F00537,F00513,F00514,F00286,F00287,F00288,F00556,F00557,F00558,F00540,F00542,F00566,F00567,F00534,F00535,F00536,F01276,F00531,F00532,F00533,F01437,...,F00716,F00717,F00718,F00719,F00720,F00721,F00722,F00723,F00724,F00725,F00726,F00727,F00728,F00729,F00730,F00731,F01345,F01346,F01347,F00732,F00754,F00755,F00756,F01416,F01417,F01418,F01419,F00751,F00752,F00753,F01404,F01430,F01431,F00743,F00744,F00745,F00746,F00747,F00748,F00749,F00750,F00760,F00761,F00762,F00757,F00758,F00759,F00791,F00769,F01398,F01399,F00780,F00781,F00782,F00770,F00771,F00772,F01408,F00777,F00778,F00779,F01405,F00773,F00774,F01392,F01393,F01406,F00775,F00776,F01394,F0139

In [34]:
(fred.F00002 == 53904).sum()

np.int64(0)

In [36]:
groot.query("source == 'FRED'") # .head()

,GRooTID,source,versionSource,originalID,referencesAbbreviated,references,referencesDataset,referencesAdditional,family,genus,species,infraspecific,familyTNRS,genusTNRS,speciesTNRS,infraspecificTNRS,taxonomicStatus,taxonomicInformation,group,order,growthForm,photosyntheticPathway,woodiness,mycorrhizalAssociationType,mycorrhizalAssociationTypeFungalRoot,nitrogenFixationNodDB,abilityToGrownClonallyCloPla,budBearingOrganCloPla,vitality,measurementProvenance,measurementTreatments,measurementMethod,year,yearBegin,yearEnd,ageStand,agePlant,locationID,location,decimalLatitude,decimalLongitud,climaticInformation,biomesKoeppen,biomesKoeppenGroup,temperatureColdestMonth,temperatureWarmestMonth,meanAnnualTemperature,meanAnnualPrecipitation,elevation,slope,soilpH,soilTexture,soilCarbon,soilNitrogen,soilPhosphorus,soilCarbonToNitrogen,soilBaseCationSaturation,soilCationExchangeCapacity,soilOrganicMatter,soilWaterGravimetric,soilWaterVolumetric,belowgroundEntities,belowgroundEntitiesOrderClassification,belowgroundEntitiesOrder,belowgroundEntitiesOrderMin,belowgroundEntitiesOrderMax,belowgroundEntitiesFunctional,belowgroundEntitiesDiameterMin,belowgroundEntitiesDiameterMax,traitName,traitValue,errorRiskEntries,errorRisk
0,1,FRED,2.0,33116.0,Aaltonen 1920,Aaltonen VT. 1920. Abreiten Der Forstwissensch...,"Fan Y, Miguez-Macho G, Jobbagy EG, Jackson RB,...",NaN,NaN,Betula,NaN,NaN,Betulaceae,Betula,NaN,NaN,Accepted,1.00/tpl/,NaN,Fagales,shrub,NaN,woody,NaN,EcM,NaN,NaN,NaN,unspecified,field,NaN,excavation,NaN,NaN,NaN,NaN,NaN,Pitkan palot,"Lapland, Sodankyla, N. Finland. boreal forest",67.853600,26.6652,boreal,Dfc,Continental,NaN,NaN,NaN,NaN,228.0,NaN,NaN,stone-free heath sand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rooting_depth,0.85,NaN,NaN
1,2,FRED,2.0,33115.0,Aaltonen 1920,Aaltonen VT. 1920. Abreiten Der Forstwissensch...,"Fan Y, Miguez-Macho G, Jobbagy EG, Jackson RB,...",NaN,NaN,Picea,NaN,NaN,Pinaceae,Picea,NaN,NaN,Accepted,1.00/tpl;usda/,NaN,Pinales,tree,NaN,woody,NaN,EcM,NaN,NaN,NaN,unspecified,field,NaN,excavation,NaN,NaN,NaN,NaN,NaN,Ojalahden Lehto,"Lapland, Sodankyla, N. Finland. boreal forest",66.637300,25.3457,boreal,Dfc,Continental,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rooting_depth,0.90,NaN,NaN
2,3,FRED,2.0,33113.0,Aaltonen 1920,Aaltonen VT. 1920. Abreiten Der Forstwissensch...,"Fan Y, Miguez-Macho G, Jobbagy EG, Jackson RB,...",NaN,NaN,Pinus,NaN,NaN,Pinaceae,Pinus,NaN,NaN,Accepted,1.00/tpl/,NaN,Pinales,tree,NaN,woody,NaN,EcM,NaN,NaN,NaN,unspecified,field,NaN,excavation,NaN,NaN,NaN,NaN,NaN,Pitkan palot,"Lapland, Sodankyla, N. Finland. boreal forest",67.853600,26.6652,boreal,Dfc,Continental,NaN,NaN,NaN,NaN,228.0,NaN,NaN,stone-free heath sand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rooting_depth,1.00,NaN,NaN
3,4,FRED,2.0,33112.0,Aaltonen 1920,Aaltonen VT. 1920. Abreiten Der Forstwissensch...,"Fan Y, Miguez-Macho G, Jobbagy EG, Jackson RB,...",NaN,NaN,Pinus,NaN,NaN,Pinaceae,Pinus,NaN,NaN,Accepted,1.00/tpl/,NaN,Pinales,tree,NaN,woody,NaN,EcM,NaN,NaN,NaN,unspecified,field,NaN,excavation,NaN,NaN,NaN,NaN,NaN,Kaunisvaaran kangas,"Lapland, Sodankyla, N. Finland. boreal forest",67.247000,26.8505,boreal,Dfc,Continental,NaN,NaN,NaN,NaN,170.0,NaN,NaN,"sandy, stone",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rooting_depth,0.98,NaN,NaN
4,5,FRED,2.0,33114.0,Aaltonen 1920,Aaltonen VT. 1920. Abreiten Der Forstwissensch...,"Fan Y, Miguez-Macho G, Jobbagy EG, Jackson RB,...",NaN,NaN,Pinus,NaN,NaN,Pinaceae,Pinus,NaN,NaN,Accepted,1.00/tpl/,NaN,Pinales,tree,NaN,woody,NaN,EcM,NaN,NaN,NaN,unspecified,field,NaN,excavation,NaN,NaN,NaN,NaN,NaN,Rakkiluoston kangas,"Lapland, Sodankyla, N. Finland. boreal forest",67.152200,26.9400,boreal,Dfc,Continental,NaN,NaN,NaN,NaN,238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rooting_depth,0.56,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,..

In [37]:
groot.query("source == 'FRED'").shape

(60282, 73)

In [39]:
# fred.loc[:, colnames[colnames.str.contains("root")]].isna().mean()

In [45]:
# common references between FRED and GRooT
np.intersect1d(fred.F00003.dropna()[9:], groot.referencesAbbreviated).size

914

In [48]:
# 
np.unique(fred.F00003.dropna()[9:], return_counts=True)

(array(['Aaltonen 1920', 'Abaimov et al. 1997', 'Abbott et al. 1991', ...,
        'van Wijk et al, 2003', 'van Wyk 1963', 'van der Wal et al, 2001'],
       shape=(1596,), dtype=object),
 array([5, 2, 6, ..., 1, 2, 2], shape=(1596,)))

In [50]:
np.unique(groot.referencesAbbreviated, return_counts=True)

(array([' Akhmetzhanova et al. 2012', 'Aaltonen 1920',
        'Abbott et al. 1991', ..., 'van Praag et al 1988',
        'van Rees and Comerford 1986', 'van Wyk 1963'],
       shape=(1607,), dtype=object),
 array([5708,    5,    6, ...,   42,    2,    1], shape=(1607,)))

In [52]:
np.unique(groot.query("source == 'FRED'").referencesAbbreviated, return_counts=False).size

919

In [38]:
# meta.name[meta.name.str.contains("order")]
# meta.loc[21, :] - F00056

fred.loc[:, "F00056"].max()

np.float64(10.0)

In [39]:
fred.loc[:, ["F00017", "F00018", "F00019", "F00056"]].loc[fred.loc[:, "F00056"] == 10, :]

,F00017,F00018,F00019,F00056
27685,NaN,Pinus,sylvestris,10.0
27722,NaN,Pinus,sylvestris,10.0
27741,NaN,Pinus,sylvestris,10.0
27759,NaN,Pinus,sylvestris,10.0
27785,NaN,Pinus,sylvestris,10.0


In [46]:
fred.query("F00018=='Pinus' & F00019=='sylvestris'").loc[:, ["F00056", "F00679"]].dropna()

,F00056,F00679
27660,1.0,0.281180
27661,2.0,0.279850
27662,3.0,0.319383
27663,4.0,0.411080
27664,5.0,0.483160
27665,6.0,0.538720
27666,7.0,0.601240
27667,8.0,0.646450
27668,1.0,0.299800
27669,2.0,0.294217


In [10]:
# no other species in genus Zea huh????
fred.query("F00018==\"Zea\"").loc[:, ["F00043", "F00018", "F00019"]].drop_duplicates()

,F00043,F00018,F00019
14389,C4,Zea,mays


In [23]:
# find the genera with most trait records

with pd.option_context("display.max_rows", None):
    print(fred.groupby(["F00018", "F00019"]).F00001.count())

F00018              F00019                   
Abelia              biflora                         1
Abies               alba                           72
                    amabilis                       48
                    arizonica                       1
                    balsamea                       23
                    balsamifera                     9
                    cephalonica                     1
                    cilicica                        1
                    concolor                        9
                    faxoniana                      35
                    firma                           2
                    fraseri                         1
                    gamblei                         1
                    grandis                         2
                    holophylla                      1
                    homolepis                       2
                    koreana                         1
                    lasiocarpa      

In [16]:
# CAM???

fred.loc[:, ["F00018", "F00019", "F00043"]].dropna(subset=["F00018", "F00043"]).groupby("F00018").F00043.unique()[
    fred.loc[:, ["F00018", "F00019", "F00043"]].dropna(subset=["F00018", "F00043"]).groupby("F00018").F00043.unique().apply(len) > 1]

F00018
Artemisia                [C3, C4]
Aster                [C3, C3/CAM]
Atriplex          [C4, C3, C3/C4]
Chenopodium           [C3, C3/C4]
Corylus              [C3, C3/CAM]
Cyperus                  [C4, C3]
Euphorbia         [C4, C3, C3/C4]
Haloxylon                [C4, C3]
Heliotropium             [C4, C3]
Hierochloe            [C3/C4, C3]
Horaninovia           [C4, C3/C4]
Opuntia                 [CAM, C3]
Portulaca            [C4, C4/CAM]
Potamogeton              [C3, C4]
Salsola           [C4, C3, C3/C4]
Sedum           [CAM, C3, C3/CAM]
Setaria                  [C3, C4]
Suaeda            [C4, C3/C4, C3]
Taxus                [C3, C3/CAM]
Verbascum             [C3/C4, C3]
Yucca                [C3, C3/CAM]
Name: F00043, dtype: object